In [4]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import time
import pydde as d

In [5]:
#Parameters
samplenum = 50
epochs = 200
input_size = 3
output_size = 3
learning_rate = 0.01
time_length = 60; #seconds

In [6]:
# Generate simulation
sim = d.PySimSeq('test2.sim', 60)
yseq = sim.compute(sim.p)
f = sim.f(sim.y, sim.ydot, sim.yddot, sim.p)
df = sim.df_dp(sim.y, sim.ydot, sim.yddot, sim.p)
dy = sim.dy_dp(sim.y, sim.ydot, sim.yddot, sim.p)

## Building the custon Simulation Activation Function

In [7]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        yTraj_pred = sim.compute(p)
        y_pred = torch.tensor(yTraj_pred[-3:]).double()
        #print(f'y_pred: {y_pred.dtype}')
        
        ctx.save_for_backward(input)
        
        return y_pred, input
    
    @staticmethod
    def backward(ctx, grad_output, input):
        #print(grad_output.shape)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        yTraj_pred = sim.compute(p)
        dy_dp = sim.dy_dp(sim.y, sim.ydot, sim.yddot, p)
        dy_dp = dy_dp[-3:, :]
        grad_output = grad_output.unsqueeze(0)
        
        grad_input = torch.tensor(dy_dp).t().mm(grad_output.t()).t().double()
        return grad_input, None

Simulate = Simulate.apply

## FE Test

In [14]:
# Error for whole simulation
from numpy import linalg as LA

FE = 1e-6
yseq = sim.compute(sim.p)
p = sim.p
p = torch.tensor(p, requires_grad = True)
y, p0 = Simulate(p)
test = sum(y)
test.backward()
dy_dp = p.grad
print(p.grad)
dy_dp_FD = np.zeros((1,len(sim.p)))

for i in range(len(sim.p)):
    dp= np.zeros(len(sim.p))
    dp[i] = FE
    dp = torch.tensor(dp)
    y_p, pp = Simulate(p + dp)
    y_m, pm = Simulate(p - dp)
    y_p = y_p.detach().numpy()
    y_m = y_m.detach().numpy()
    dy_dp_FD[0, i] = sum((y_p - y_m) / (2* FE))

print(dy_dp_FD)

#dy_dp_FD = dy_dp_FD.detach().numpy()
dy_dp = dy_dp.detach().numpy()
err = LA.norm(dy_dp_FD - dy_dp)
print(err)

tensor([ 6.1626e-04,  6.2852e-08,  6.1627e-04,  3.1267e-03,  2.7846e-07,
         3.1267e-03,  5.6354e-03,  4.9105e-07,  5.6355e-03,  8.1357e-03,
         6.9829e-07,  8.1358e-03,  1.0621e-02,  8.9795e-07,  1.0621e-02,
         1.3083e-02,  1.0879e-06,  1.3083e-02,  1.5517e-02,  1.2659e-06,
         1.5517e-02,  1.7915e-02,  1.4303e-06,  1.7916e-02,  2.0271e-02,
         1.5790e-06,  2.0271e-02,  2.2578e-02,  1.7107e-06,  2.2579e-02,
         2.4830e-02,  1.8237e-06,  2.4830e-02,  2.7020e-02,  1.9169e-06,
         2.7021e-02,  2.9143e-02,  1.9893e-06,  2.9144e-02,  3.1192e-02,
         2.0400e-06,  3.1193e-02,  3.3162e-02,  2.0686e-06,  3.3162e-02,
         3.5046e-02,  2.0748e-06,  3.5047e-02,  3.6841e-02,  2.0583e-06,
         3.6842e-02,  3.8540e-02,  2.0195e-06,  3.8541e-02,  4.0138e-02,
         1.9588e-06,  4.0139e-02,  4.1632e-02,  1.8767e-06,  4.1633e-02,
         4.3016e-02,  1.7743e-06,  4.3017e-02,  4.4287e-02,  1.6526e-06,
         4.4288e-02,  4.5440e-02,  1.5129e-06,  4.5

In [9]:
from numpy import linalg as LA

#Calculate dy_dp with FE
FE = 1e-6
dy_dp = sim.dy_dp(sim.y, sim.ydot, sim.yddot, sim.p)
dy_dp_FD = np.zeros((len(sim.p),len(sim.p)))

for i in range(len(sim.p)):
    dp= np.zeros(len(sim.p))
    dp[i] = FE
    pos = sim.p + dp
    y_p = sim.compute(pos)
    y_m = sim.compute(sim.p - dp)
    dy_dp_FD[:, i] = (y_p - y_m) / (2* FE)
err = LA.norm(dy_dp_FD - dy_dp)
print(err)


0.0002908451455664775


## Grad Check

In [12]:
from torch.autograd import gradcheck

# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
p = sim.p
p = torch.tensor(p, requires_grad = True)
input = p.double()
test = gradcheck(Simulate, input, eps=1e-6, atol=1e-2)
print(test), nondet_to, nondet_tol=0.0)

SyntaxError: invalid syntax (<ipython-input-12-7017e70665cd>, line 10)